In [1]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)
ws = Workspace.from_config()

1.12.0
WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [2]:
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

rest_endpoint1 = "https://northeurope.api.azureml.ms/pipelines/v1.0/subscriptions/ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9/resourceGroups/LUZHAN/providers/Microsoft.MachineLearningServices/workspaces/luzhan-north-europe/PipelineRuns/PipelineSubmit/e9cfeeae-52db-4c43-b1ba-220a66a5f2cf"

# specify the param when running the pipeline
response = requests.post(rest_endpoint1, 
                         headers=aad_token, 
                         json={"ExperimentName": "Test_Model_Register_Exp",
                               "RunSource": "SDK",
                               "ParameterAssignments": {"Dataset1": "3444e16a-245c-4dec-a175-16890e36c665"}})

In [3]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception('Received bad response from the endpoint: {}\n'
                    'Response Code: {}\n'
                    'Headers: {}\n'
                    'Content: {}'.format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  04709bcb-328c-4d80-a362-22fa128a83d3


In [9]:
from azureml.pipeline.core.run import PipelineRun
run = PipelineRun.get(ws, run_id)
run.wait_for_completion()

train_model_run = run.find_step_run(name="azureml://Train Model")[0]

PipelineRunId: 04709bcb-328c-4d80-a362-22fa128a83d3
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Test_Model_Register_Exp/runs/04709bcb-328c-4d80-a362-22fa128a83d3?wsid=/subscriptions/ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9/resourcegroups/LUZHAN/workspaces/luzhan-north-europe

PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': '04709bcb-328c-4d80-a362-22fa128a83d3', 'status': 'Completed', 'startTimeUtc': '2020-08-31T09:02:15.643659Z', 'endTimeUtc': '2020-08-31T09:15:47.26955Z', 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'HTTP', 'azureml.parameters': '{"Dataset1":"3444e16a-245c-4dec-a175-16890e36c665"}', 'azureml.pipelineid': 'e9cfeeae-52db-4c43-b1ba-220a66a5f2cf'}, 'inputDatasets': [], 'logFiles': {'logs/azureml/executionlogs.txt': 'https://luzhannortheur0420725019.blob.core.windows.net/azureml/ExperimentRun/dcid.04709bcb-328c-4d80-a362-22fa128a83d3/logs/azureml/executionlogs.txt?sv=2019-02-02&sr=b&

In [12]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
local_path = './model-dataset'
target_path = 'test/model-data'

port_output = train_model_run.get_output_data('Trained_model')
model_file = port_output.download(local_path, overwrite=True)
datastore.upload(src_dir = local_path, target_path = target_path)

Uploading an estimated of 9 files
Uploading ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\conda_env.yaml
Uploading ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\data.ilearner
Uploading ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\data.metadata
Uploading ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\data_type.json
Uploading ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\model_spec.yaml
Uploaded ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\conda_env.yaml, 1 files out of an estimated total of 9
Uploading ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\score.py
Uploaded ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\data.ilearner, 2 files out of an estimated total of 9
Uploading ./model-dataset\azureml\ea4b1e4f-bf76-471b-a0ce-324fe340e9f6\Trained_model\_meta.yaml


$AZUREML_DATAREFERENCE_c769d90eb4bc4405add385e9db54a058

In [14]:
datastore_path = [(datastore, target_path)]
model-dataset = Dataset.File.from_files(path=datastore_path)
registered-model-dataset = model-dataset.register(workspace=ws, name='registered-model-from-sample1')

SyntaxError: can't assign to operator (<ipython-input-14-b7ed1feb62bb>, line 2)